In [1]:
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import silhouette_score

from IPython.display import Audio, display
def allDone():
    urL = 'https://www.wavsource.com/snds_2020-10-01_3728627494378403/animals/cat_meow2.wav'
    display(Audio(url=urL, autoplay=True))
# allDone()

random.seed(666)

In [2]:
# train_motifs = np.genfromtxt('data_dev/train_motifs.csv',dtype='U')
# train_motifxFamMatrix = np.genfromtxt('data_dev/train_motifxFamMatrix.csv',delimiter=',',dtype=int)
# test_motifs = np.genfromtxt('data_dev/new_test_motifs.csv',dtype='U')
# test_motifxFamMatrix = np.genfromtxt('data_dev/new_test_motifxFamMatrix.csv',delimiter=',',dtype=int)

train_motifs = np.genfromtxt('data_dev/LEGACY/train_motifs.csv',dtype='U')
train_motifxFamMatrix = np.genfromtxt('data_dev/LEGACY/train_motifxFamMatrix.csv',delimiter=',',dtype=int)
test_motifs = np.genfromtxt('data_dev/LEGACY/new_test_motifs.csv',dtype='U')
test_motifxFamMatrix = np.genfromtxt('data_dev/LEGACY/new_test_motifxFamMatrix.csv',delimiter=',',dtype=int)

fams = np.genfromtxt('data_dev/fams.csv',dtype='U')

In [3]:
from sklearn.model_selection import train_test_split

all_motifs = np.hstack([train_motifs,test_motifs])
all_motifxFamMatrix = np.vstack([train_motifxFamMatrix,test_motifxFamMatrix])

print(train_motifxFamMatrix.shape)
print(test_motifxFamMatrix.shape)      

(6449, 8)
(853, 8)


In [13]:
run = '02'
embedding = np.genfromtxt('data_dev/LEGACY/protvec_embedding.csv',delimiter=',',dtype=float)
# embedding = np.genfromtxt('MODELS_siam/LEGACY/emb_%s_embedding.csv' % run,delimiter=',',dtype=float)

train_embedding = embedding[ :len(train_motifs) ]
test_embedding = embedding[ len(train_motifs): ]

print(train_embedding.shape)
print(test_embedding.shape)

(6449, 1300)
(853, 1300)


In [14]:
#### SILHOUETTE SCORES #####

allSilhouettes = []
for i,fam in enumerate(fams):
    silhouette = silhouette_score(test_embedding, test_motifxFamMatrix.T[i], 
                                  metric='euclidean', sample_size=None, random_state=None)
    allSilhouettes.append(silhouette)
allSilhouettes_array = np.array((allSilhouettes))

# print( np.mean(allSilouettes_array), np.median(allSilouettes_array),
#        fams,[round(x,6) for x in allSilouettes] )

print("mean silhouette:", np.mean(allSilhouettes_array).round(3))
print("median silhouette", np.median(allSilhouettes_array).round(3))

mean silhouette: 0.005
median silhouette 0.005


In [15]:
def getKnnResults(fIdx, myAverage):
    
    ## best k is num of instances ^ 1/2
    ## https://stackoverflow.com/questions/18110951/how-to-determine-k-value-for-the-k-nearest-neighbours-algorithm-for-a-matrix-in
    k = int(len(embedding)**(1/2))
    
    X_tr = train_embedding 
    X_te = test_embedding
    y_tr = train_motifxFamMatrix.T[fIdx]
    y_te = test_motifxFamMatrix.T[fIdx]
    
    kNN = KNeighborsClassifier(n_neighbors=k)
    kNN.fit(X_tr, y_tr)

    y_pred = kNN.predict(X_te)
    y_score = kNN.predict_proba(X_te)

    precision, recall, fscore, support = score(y_te,y_pred,average=myAverage)
    return (precision, recall, fscore, support, y_score)

In [16]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_recall_fscore_support as score

thisY_score = [] 
for fIdx,fam in enumerate(fams):
    precision, recall, fscore, support, myY_score = (getKnnResults(fIdx, "macro"))
    thisY_score.append(myY_score)

/home/k4thryn/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/k4thryn/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/k4thryn/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/k4thryn/anaconda3/lib/

In [17]:
thisY_score = np.array((thisY_score))
thisY_score.shape

y_score = thisY_score[:,:,1].T
y_test = test_motifxFamMatrix

In [18]:
from sklearn.metrics import roc_curve, auc
from scipy import interp
import numpy as np

print("AUROC results:")
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(len(fams)):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
    print(fams[i],roc_auc[i].round(3))


fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(len(fams))]))

mean_tpr = np.zeros_like(all_fpr)
for i in range(len(fams)):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])
mean_tpr /= len(fams)

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

print("macro auroc:",roc_auc["macro"].round(3),
      "\nmicro auroc:",roc_auc["micro"].round(3))

AUROC results:
PKA 0.816
AKT 0.832
CDK 0.857
MAPK 0.888
SRC 0.979
CK2 0.872
PKC 0.83
PIKK 0.81
macro auroc: 0.861 
micro auroc: 0.851


<ipython-input-18-d7d4b22900b0>:21: DeprecationWarning: scipy.interp is deprecated and will be removed in SciPy 2.0.0, use numpy.interp instead
  mean_tpr += interp(all_fpr, fpr[i], tpr[i])


In [19]:
from itertools import chain
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score

precision = dict()
recall = dict()
average_precision = dict()

print("AVERAGE Precision (AP) results")
for i in range(len(fams)):
    precision[i], recall[i], _ = precision_recall_curve(y_test[:, i],y_score[:, i])
    average_precision[i] = average_precision_score(y_test[:, i], y_score[:, i])
    print(fams[i],average_precision[i].round(3))


average_precision["micro"] = average_precision_score(y_test, y_score,average="micro")
average_precision["macro"] = average_precision_score(y_test, y_score, average="macro")
print("avg AP (macro):",average_precision["macro"].round(3),
      "\navg AP (micro):",average_precision["micro"].round(3))

AVERAGE Precision (AP) results
PKA 0.625
AKT 0.421
CDK 0.41
MAPK 0.688
SRC 0.906
CK2 0.59
PKC 0.638
PIKK 0.396
avg AP (macro): 0.584 
avg AP (micro): 0.551


In [20]:
# allDone()

In [21]:
print("mean silhouette:", np.mean(allSilhouettes_array).round(3))
print("median silhouette", np.median(allSilhouettes_array).round(3))

print("----------------\n")

print("AUROC results:")
for i in range(len(fams)):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
    print(fams[i],roc_auc[i].round(3))
print("macro auroc:",roc_auc["macro"].round(3),
      "\nmicro auroc:",roc_auc["micro"].round(3))

print("\n----------------\n")

print("AVERAGE Precision (AP) results")
for i in range(len(fams)):
    precision[i], recall[i], _ = precision_recall_curve(y_test[:, i],y_score[:, i])
    average_precision[i] = average_precision_score(y_test[:, i], y_score[:, i])
    print(fams[i],average_precision[i].round(3))
average_precision["micro"] = average_precision_score(y_test, y_score,average="micro")
average_precision["macro"] = average_precision_score(y_test, y_score, average="macro")
print("avg AP (macro):",average_precision["macro"].round(3),
      "\navg AP (micro):",average_precision["micro"].round(3))

mean silhouette: 0.005
median silhouette 0.005
----------------

AUROC results:
PKA 0.816
AKT 0.832
CDK 0.857
MAPK 0.888
SRC 0.979
CK2 0.872
PKC 0.83
PIKK 0.81
macro auroc: 0.861 
micro auroc: 0.851

----------------

AVERAGE Precision (AP) results
PKA 0.625
AKT 0.421
CDK 0.41
MAPK 0.688
SRC 0.906
CK2 0.59
PKC 0.638
PIKK 0.396
avg AP (macro): 0.584 
avg AP (micro): 0.551
